In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 필요한 라이브러리 호출
import torch
import random

In [3]:
# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 랜덤시드 고정
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [14]:
# 학습 파라미터 설정
learning_rate = 0.001
training_epochs = 5

In [15]:
# 데이터 불러오기
X_train = pd.read_csv("../input/2021-ai-w6-p2/train.csv").iloc[:, :-1]
y_train = pd.read_csv("../input/2021-ai-w6-p2/train.csv").iloc[:, -1]
X_test = pd.read_csv("../input/2021-ai-w6-p2/X_test.csv").iloc[:, 1:]

In [16]:
# 데이터 전처리
from sklearn.preprocessing import StandardScaler
std =StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

In [17]:
# Pandas to Tensor 변환 함수
def pd_to_tensor(pandas_data):
  to_numpy = np.array(pandas_data)
  return torch.FloatTensor(to_numpy)

In [18]:
# 판다스 데이터프레임 형태의 데이터를 텐서로 변환
X_train_tensor = pd_to_tensor(X_train_std).to(device)
y_train_tensor = pd_to_tensor(y_train).to(device)
X_test_tensor = pd_to_tensor(X_test_std).to(device)

In [19]:
# NN모델 설계
class NN(torch.nn.Module):
  def __init__(self):
    super(NN,self).__init__()
    self.linear1 = torch.nn.Linear(8,256,bias=True)
    self.linear2 = torch.nn.Linear(256,64,bias=True)
    self.linear3 = torch.nn.Linear(64,16,bias=True)
    self.linear4 = torch.nn.Linear(16,1,bias=True)
    self.sigmoid = torch.nn.Sigmoid()

  def forward(self,x):
    out = self.linear1(x)
    out = self.sigmoid(out)
    out = self.linear2(out)
    out = self.sigmoid(out)
    out = self.linear3(out)
    out = self.sigmoid(out)
    out = self.linear4(out)
    return out

model = NN().to(device)

In [20]:
# 손실함수와 최적화 함수
loss = torch.nn.MSELoss().to(device) # softmax 내부적으로 계산
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [21]:
# 모델 학습
model.train()
for epoch in range(training_epochs):
    total_cost = 0
    for X, Y in zip(X_train_tensor, y_train_tensor):
        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()
        # Error 합계
        total_cost += cost
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(total_cost))
print('Learning finished')

In [22]:
# 모델 평가
model.eval()
with torch.no_grad():
    prediction = model(X_test_tensor)

In [24]:
# 제출파일 만들기
submit = pd.read_csv("../input/2021-ai-w6-p2/sample_submit.csv")
submit["TARGET"] = prediction.cpu().numpy()
submit.to_csv("submit-kiy4.csv", header= True, index = False)